
# Steven R. WHITE-DMRG <a href="https://www.youtube.com/watch?v=sDcnFgfx4mw">lecture</a> solutions
Han-Ze Li XTU
## Ex. 1 diagonalize N=2 H matrix

In [11]:
H = [1/4  0.0   0.0   0.0; 
     0.0 -1/4   1/2   0.0;
     0.0  1/2  -1/4   0.0;
     0.0  0.0   0.0   1/4]

4×4 Matrix{Float64}:
 0.25   0.0    0.0   0.0
 0.0   -0.25   0.5   0.0
 0.0    0.5   -0.25  0.0
 0.0    0.0    0.0   0.25

In [12]:
using LinearAlgebra
eigen(H)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
4-element Vector{Float64}:
 -0.7499999999999994
  0.25
  0.25
  0.25
vectors:
4×4 Matrix{Float64}:
  0.0       1.0  0.0       0.0
  0.707107  0.0  0.707107  0.0
 -0.707107  0.0  0.707107  0.0
  0.0       0.0  0.0       1.0

## Ex 2. N=4 $H^{S_{total}=0}_{block}$ block martrix diagonalization

In [13]:
using LinearAlgebra

H = [0.0  0.0   0.0   0.0   0.0   0.0; 
     0.0  0.0   0.0   1/2   1/2   0.0;
     0.0  0.0   0.0   0.0   1/2   0.0;
     0.0  1/2   0.0   0.0   0.0   0.0;
     0.0  1/2   1/2   0.0   0.0   0.0;
     0.0  0.0   0.0   0.0   0.0   0.0]

eigen(H)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
6-element Vector{Float64}:
 -0.8090169943749461
 -0.30901699437494723
  0.0
  0.0
  0.3090169943749481
  0.8090169943749475
vectors:
6×6 Matrix{Float64}:
  0.0        0.0       1.0  0.0   0.0        0.0
 -0.601501   0.371748  0.0  0.0   0.371748  -0.601501
 -0.371748  -0.601501  0.0  0.0  -0.601501  -0.371748
  0.371748  -0.601501  0.0  0.0   0.601501  -0.371748
  0.601501   0.371748  0.0  0.0  -0.371748  -0.601501
  0.0        0.0       0.0  1.0   0.0        0.0

## Ex 3 & 4. N-spins Heisenberg chain

$$\hat{H}=\sum_{j=1}^{L-1}{\bf S}_j \cdot {\bf S}_{j+1}=\sum_{j=1}^{L-1}S_{j}^{z}S_{j+1}^{z}+ \frac{1}{2}(S_{j}^{+}S_{j+1}^{-}+ S_{j}^{-}S_{j+1}^{+})$$

In [48]:
using LinearAlgebra, SparseArrays, Arpack

# Pauli matrices
Sz = sparse([1/2 0.0; 0.0 -1/2])  # Sz matrix
Sp = sparse([0.0 1.0; 0.0 0.0])   # S+ matrix
Sm = sparse([0.0 0.0; 1.0 0.0])   # S- matrix

# system size
L = 10

# Define S_zz, S_pm, S_mp functions using anonymous functions
S_zz = index -> kron(kron(Sz, Sz), sparse(I, 2^(L-2), 2^(L-2)))
S_pm = index -> kron(kron(Sp, Sm), sparse(I, 2^(L-2), 2^(L-2)))
S_mp = index -> kron(kron(Sm, Sp), sparse(I, 2^(L-2), 2^(L-2)))

# Calculate the Hamiltonian
Hamiltonian = sum(S_zz(i) + 0.5 * (S_pm(i) + S_mp(i)) for i in 1:(L-1))

# Compute eigensystem
@time E, vec = eigs(Hamiltonian, nev=6, which=:SR)

# Print ground state energy
println("Ground state energy:", minimum(E))


  0.003422 seconds (177 allocations: 259.906 KiB)
Ground state energy:-6.7500000000000036


## Ex. 6 Entanglement entropy of spin chains

In [142]:
using LinearAlgebra, SparseArrays, Arpack

# Pauli matrices
Sz = sparse([1/2 0.0; 0.0 -1/2])  # Sz matrix
Sp = sparse([0.0 1.0; 0.0 0.0])   # S+ matrix
Sm = sparse([0.0 0.0; 1.0 0.0])   # S- matrix

# system size
L = 2

# Define S_zz, S_pm, S_mp functions using anonymous functions
S_zz = index -> kron(kron(Sz, Sz), sparse(I, 2^(L-2), 2^(L-2)))
S_pm = index -> kron(kron(Sp, Sm), sparse(I, 2^(L-2), 2^(L-2)))
S_mp = index -> kron(kron(Sm, Sp), sparse(I, 2^(L-2), 2^(L-2)))

# Calculate the Hamiltonian
Hamiltonian = sum(S_zz(i) + 0.5 * (S_pm(i) + S_mp(i)) for i in 1:(L-1))

# Compute eigensystem
E, vec = eigs(Hamiltonian, nev=L, which=:SR)

# Obtain ground state vector
ground_state = vec[:, 1]

# Reshape ground state vector into left and right subsystems
left_dim = 2^(L ÷ 2)
right_dim = 2^(L ÷ 2)
ground_state_left = reshape(ground_state, left_dim, right_dim)

# Perform SVD on the left subsystem
U, Σ, V = svd(ground_state_left)

# Compute the entanglement entropy
entanglement_entropy = -sum(σ^2 * log(σ^2) for σ in Σ)

# Smax
Smax = (L/2)*log.(2)

println("Se:", entanglement_entropy)
println("Se-max:", Smax)

Se:0.6931471805599454
Se-max:0.6931471805599453
